In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('problem3.csv')

# a. Calculate the pairwise covariance matrix of the data.

In [2]:
# 计算协方差矩阵
cov_matrix = data.cov()

# 打印协方差矩阵
print("成对协方差矩阵:")
print(cov_matrix)

成对协方差矩阵:
          x1        x2        x3        x4        x5
x1  1.470484  1.454214  0.877269  1.903226  1.444361
x2  1.454214  1.252078  0.539548  1.621918  1.237877
x3  0.877269  0.539548  1.272425  1.171959  1.091912
x4  1.903226  1.621918  1.171959  1.814469  1.589729
x5  1.444361  1.237877  1.091912  1.589729  1.396186


# b. Is the matrix at least positive semi-definite? Why? 

In [3]:
# 定义判断正半定的函数
def is_positive_semi_definite(matrix):
    """
    判断矩阵是否为正半定
    """
    # 检查所有特征值是否非负
    eigenvalues = np.linalg.eigvals(matrix)
    return np.all(eigenvalues >= 0), eigenvalues

# 检查协方差矩阵的正半定性
is_psd, eigenvalues = is_positive_semi_definite(cov_matrix)

if is_psd:
    print("协方差矩阵是正半定的，因为所有特征值均非负。")
else:
    print("协方差矩阵不是正半定的，因为存在负特征值。")
    print(f"特征值: {eigenvalues}")


协方差矩阵不是正半定的，因为存在负特征值。
特征值: [ 6.78670573  0.83443367 -0.31024286  0.02797828 -0.13323183]


# c. If not, find the nearest positive semi-definite matrix using Higham’s method as defined in the notes.

In [4]:
def nearest_positive_semi_definite(matrix, max_iter=100, tolerance=1e-8):
    """
    使用 Higham 方法找到最近的正半定矩阵
    """
    # 初始化
    n = matrix.shape[0]
    Y = matrix
    delta_S = np.zeros((n, n))
    for k in range(max_iter):
        # 对称化
        R = Y - delta_S
        X = (R + R.T) / 2
        
        # 特征值分解
        eigval, eigvec = np.linalg.eigh(X)
        eigval = np.maximum(eigval, 0)  # 将负特征值替换为 0
        X_psd = eigvec @ np.diag(eigval) @ eigvec.T
        
        # 更新调整量
        delta_S = X_psd - R
        Y = X_psd
        
        # 检查收敛
        if np.linalg.norm(delta_S, ord='fro') < tolerance:
            break
    return Y

# 如果矩阵不是正半定的，找到最近的正半定矩阵
if not is_psd:
    nearest_psd_matrix = nearest_positive_semi_definite(cov_matrix)
    print("最近的正半定矩阵:")
    print(nearest_psd_matrix)
    
    # 验证新的矩阵是否为正半定
    is_psd_new, eigenvalues_new = is_positive_semi_definite(nearest_psd_matrix)
    print("新的矩阵是否为正半定:", is_psd_new)
    print(f"新的矩阵特征值: {eigenvalues_new}")


最近的正半定矩阵:
[[1.61513295 1.44196041 0.89714421 1.78042572 1.43379434]
 [1.44196041 1.34696791 0.58508635 1.55455193 1.21140918]
 [0.89714421 0.58508635 1.29891578 1.11595578 1.07669234]
 [1.78042572 1.55455193 1.11595578 1.98316488 1.62137332]
 [1.43379434 1.21140918 1.07669234 1.62137332 1.40493616]]
新的矩阵是否为正半定: True
新的矩阵特征值: [6.78670573e+00+0.0000000e+00j 8.34433668e-01+0.0000000e+00j
 2.79782808e-02+0.0000000e+00j 4.55180126e-17+1.7281321e-16j
 4.55180126e-17-1.7281321e-16j]
